<a href="https://colab.research.google.com/github/partizanos/advanced_image_processing/blob/master/tp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tp4 image processing JPEG -- DIMITRIS PROIOS

## Exercise 1 (a) Implement by yourself the JPEG compression algorithm described above.

In [346]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import math

response = requests.get("https://raw.githubusercontent.com/partizanos/advanced_image_processing/master/TP/TP4/test.png")
im = Image.open(BytesIO(response.content))

# test image used in Rafael Gonzalez book
# im = test_image = [[
#     [52, 55, 61, 66, 70, 61, 64, 73],
#     [63, 59, 66, 90, 109, 85, 69, 72],
#     [62, 59, 68, 113, 144, 104, 66, 73],
#     [63, 58, 71, 122, 154, 106, 70, 69],
#     [67, 61, 68, 104, 126, 88, 68, 70],
#     [79, 65, 60, 70, 77, 63, 58, 75],
#     [85, 71, 64, 59, 55, 61, 65, 83],
#     [87, 79, 69, 68, 65, 76, 78, 94]
# ],
#     [
#     [52, 55, 61, 66, 70, 61, 64, 73],
#     [63, 59, 66, 90, 109, 85, 69, 72],
#     [62, 59, 68, 113, 144, 104, 66, 73],
#     [63, 58, 71, 122, 154, 106, 70, 69],
#     [67, 61, 68, 104, 126, 88, 68, 70],
#     [79, 65, 60, 70, 77, 63, 58, 75],
#     [85, 71, 64, 59, 55, 61, 65, 83],
#     [87, 79, 69, 68, 65, 76, 78, 94]
# ],
#     [
#     [52, 55, 61, 66, 70, 61, 64, 73],
#     [63, 59, 66, 90, 109, 85, 69, 72],
#     [62, 59, 68, 113, 144, 104, 66, 73],
#     [63, 58, 71, 122, 154, 106, 70, 69],
#     [67, 61, 68, 104, 126, 88, 68, 70],
#     [79, 65, 60, 70, 77, 63, 58, 75],
#     [85, 71, 64, 59, 55, 61, 65, 83],
#     [87, 79, 69, 68, 65, 76, 78, 94]
# ]]

img = np.array(im)
print(img.shape)
print(len(img.shape))
M, N, _ = img.shape
max_M = int(math.modf(M/8)[1] *8)
max_N = int(math.modf(N/8)[1] *8)
crop = lambda img:img[:max_M, :max_N, :]
img = crop(img )


(870, 1296, 3)
3


1. RGB color space to YCbCr color space conversion:

In [0]:
def extract_colors(img):
  R_DIMENSION = img[:, :, 0]
  G_DIMENSION = img[:, :, 1]
  B_DIMENSION = img[:, :, 2]
  return R_DIMENSION, G_DIMENSION, B_DIMENSION


def YCbCr(R_DIMENSION,G_DIMENSION,B_DIMENSION): 
  Y = np.add(np.add(0.299 * R_DIMENSION, 0.587 * G_DIMENSION), 0.114 * B_DIMENSION)
  Cb = - 0.1687 * R_DIMENSION - 0.3313 * G_DIMENSION + 0.5 * B_DIMENSION + 128
  Cr = 0.5 * R_DIMENSION - 0.4187 * G_DIMENSION - 0.0813 * B_DIMENSION + 128
  R_DIMENSION = Y + 1.402 * (Cr - 128)
  G_DIMENSION = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
  B_DIMENSION = Y + 1.772 * (Cb - 128)
  return Y, Cb, Cr, R_DIMENSION, G_DIMENSION, B_DIMENSION

Y, Cb, Cr, R_DIMENSION, G_DIMENSION, B_DIMENSION = YCbCr(*extract_colors(img))

2. Keep the Y component.
Downsample the Cb and Cr components in 2 times.
Apply DCT compression to each component.

In [348]:
def downsample_image(img, skip):
    return img[::skip,::skip]

d_cb  = downsample_image(Cb, 2)
d_cb  = downsample_image(d_cb, 2)
d_cr  = downsample_image(Cr, 2)
d_cr  = downsample_image(d_cr, 2)
Y.shape

(864, 1296)

3. Preprocessing for DCT transformation

3.1 Split the image into 8 × 8 non-overlapping blocks

In [349]:
global_mean = 128
number_octads = int(max_M* max_N / 64)
number_octads, Y.shape, max_M, max_N

(17496, (864, 1296), 864, 1296)

3.2 In each block subtract global mean computed as 2 k−1 , where k is the number of gray
levels in the image

In [350]:
Y_r = Y.reshape((number_octads, 8,8))
Cb_r = Cb.reshape((number_octads, 8,8))
Cr_r = Cr.reshape((number_octads, 8,8))

#test
Y_r[0] = [
    [52, 55, 61, 66, 70, 61, 64, 73],
    [63, 59, 66, 90, 109, 85, 69, 72],
    [62, 59, 68, 113, 144, 104, 66, 73],
    [63, 58, 71, 122, 154, 106, 70, 69],
    [67, 61, 68, 104, 126, 88, 68, 70],
    [79, 65, 60, 70, 77, 63, 58, 75],
    [85, 71, 64, 59, 55, 61, 65, 83],
    [87, 79, 69, 68, 65, 76, 78, 94]
]
Y_r.shape, Y_r[0]

((17496, 8, 8), array([[ 52.,  55.,  61.,  66.,  70.,  61.,  64.,  73.],
        [ 63.,  59.,  66.,  90., 109.,  85.,  69.,  72.],
        [ 62.,  59.,  68., 113., 144., 104.,  66.,  73.],
        [ 63.,  58.,  71., 122., 154., 106.,  70.,  69.],
        [ 67.,  61.,  68., 104., 126.,  88.,  68.,  70.],
        [ 79.,  65.,  60.,  70.,  77.,  63.,  58.,  75.],
        [ 85.,  71.,  64.,  59.,  55.,  61.,  65.,  83.],
        [ 87.,  79.,  69.,  68.,  65.,  76.,  78.,  94.]]))

In [351]:
Y_r = Y_r - global_mean
Cb_r = Cb_r - global_mean
Cr_r = Cr_r - global_mean
Y_r.shape, Y_r[0]

((17496, 8, 8), array([[-76., -73., -67., -62., -58., -67., -64., -55.],
        [-65., -69., -62., -38., -19., -43., -59., -56.],
        [-66., -69., -60., -15.,  16., -24., -62., -55.],
        [-65., -70., -57.,  -6.,  26., -22., -58., -59.],
        [-61., -67., -60., -24.,  -2., -40., -60., -58.],
        [-49., -63., -68., -58., -51., -65., -70., -53.],
        [-43., -57., -64., -69., -73., -67., -63., -45.],
        [-41., -49., -59., -60., -63., -52., -50., -34.]]))

Step 4. DCT transformation per block: T (u, v)

In [352]:
from scipy import fftpack 

Y_dct = fftpack.dct(Y_r)
Cb_dct = fftpack.dct(Cb_r)
Cr_dct = fftpack.dct(Cr_r)
Y_dct[0]

array([[-1044.        ,   -57.72015737,   -22.62145031,   -26.96553606,
           28.28427125,   -12.33157164,    -2.87575832,     9.65275291],
       [ -822.        ,   -62.80862838,  -135.86001805,    47.31686293,
           77.78174593,   -29.39383609,    -6.48502085,    13.27377425],
       [ -670.        ,   -85.31450748,  -261.39884858,    89.50082767,
          134.35028843,   -64.08917035,    -6.53008144,     4.42887148],
       [ -622.        ,   -83.10039098,  -303.58028174,    98.91599692,
          145.66399692,   -49.9983738 ,   -19.67263434,    15.55998681],
       [ -744.        ,   -48.33206974,  -192.50649839,    61.4189483 ,
          115.96551211,   -33.99070329,   -21.28962366,    16.50320918],
       [ -954.        ,    13.42217091,    12.93431346,    17.58318733,
           77.78174593,   -22.09754857,     5.35756805,     2.52491557],
       [ -962.        ,    18.79492044,   108.19802502,    -9.34447896,
           29.69848481,    -1.7248436 ,    21.00446098,   

Step 5. Block coefficients quantization:

$T(u,v) = round \frac{T(u,v)} {Z(u,v)}  $

In [353]:
Z = [
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 28, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
]

T_Y = np.round(Y_dct/Z)
T_cb = np.round(Cb_dct/Z)
T_cr = np.round(Cr_dct/Z)
# T_Y = Y_dct/Z
# T_cb = Cb_dct/Z
# T_cr = Cr_dct/Z

T_cr.shape, T_cr[0, :]

((17496, 8, 8), array([[-2.,  1.,  1.,  0.,  0., -0., -0., -0.],
        [-1., -1.,  0., -0.,  0.,  0.,  0., -0.],
        [-0.,  1.,  0.,  0., -0., -0., -0., -0.],
        [ 1., -2.,  1., -0.,  0., -0.,  0., -0.],
        [ 1., -1.,  1.,  0.,  0.,  0.,  0., -0.],
        [ 2., -1.,  1., -0.,  0., -0.,  0., -0.],
        [ 3.,  1.,  1.,  0., -0.,  0.,  0., -0.],
        [ 1., -0.,  0.,  0.,  0., -0., -0., -0.]]))

Step 6. Symbols encoding:
6.1  Zig-zag scanning


In [0]:
# https://rosettacode.org/wiki/Zig-zag_matrix#Python
import operator

def zigzag(n):
    '''zigzag rows'''
    def compare(xy):
        x, y = xy
        return (x + y, -y if (x + y) % 2 else y)
      
    xs = range(n)
    
    return {index: n for n, index in enumerate(sorted(
        ((x, y) for x in xs for y in xs),
        key=compare
    ))}


x = zigzag(8)
zig_zag_indices = sorted(x.items(), key=operator.itemgetter(1))
zig_zag_indices = [ i[0] for i in zig_zag_indices]

In [0]:
# zig zag representation
Y_zigzag = []
cb_zigzag = []
cr_zigzag = []
for block in range(T_Y.shape[0]):
  for i in zig_zag_indices:
    Y_zigzag.append(T_Y[block][i])
    cb_zigzag.append(T_cb[block][i])
    cr_zigzag.append(T_cr[block][i])
# Y_zigzag = [T_Y[block][i] for i in zig_zag_indices for block in range(T_Y.shape[0])]
# Cb_zigzag =  [[T_cb[block][i]for i in zig_zag_indices] for block in range(T_cb.shape[0])]
# Cr_zigzag =  [[T_cr[block][i]for i in zig_zag_indices] for block in range(T_cr.shape[0])]

# TEST 
# Y_zigzag[:3], [T_Y[0][0,0],T_Y[0][0,1],T_Y[0][1,0]], T_Y[0]
assert Y_zigzag[:3][0] == T_Y[0][0,0]
assert Y_zigzag[:3][1] == T_Y[0][0,1]
assert Y_zigzag[:3][2] == T_Y[0][1,0]
Y_zigzag = np.asarray(Y_zigzag)
cb_zigzag = np.asarray(cb_zigzag)
cr_zigzag = np.asarray(cr_zigzag)


In [0]:
# merge blocks
Y_merged = Y_zigzag.reshape((max_M, max_N))
Cb_merged = cb_zigzag.reshape((max_M, max_N))
Cr_merged = cb_zigzag.reshape((max_M, max_N))

6.2 Huffman coding

In [357]:
# count frequency of each color 
def count_color_freq(img):
  unique, counts = np.unique(img, return_counts=True)
  return dict(zip(unique, counts))

frequencies = [count_color_freq(chanel) for chanel in [Y_zigzag, Cb_zigzag, Cr_zigzag] ] 

total_pixels = max_N *max_M
assert max_N *max_M == sum(frequencies[0].values())

# frequencies = [ { k:v/total_pixels for k,v in f.items()} for f in frequencies]
probs = []
for f_index in range(len(frequencies)):
  probs.append({ k:v/total_pixels for k,v in frequencies[f_index].items()})
# probs[2]

{-38.0: 8.930612711476909e-07,
 -36.0: 8.930612711476909e-07,
 -35.0: 1.7861225422953817e-06,
 -33.0: 2.6791838134430725e-06,
 -32.0: 4.465306355738455e-06,
 -31.0: 1.7861225422953817e-06,
 -30.0: 6.251428898033836e-06,
 -29.0: 1.1609796524919981e-05,
 -28.0: 9.8236739826246e-06,
 -27.0: 1.3395919067215363e-05,
 -26.0: 3.125714449016918e-05,
 -25.0: 2.4112654320987653e-05,
 -24.0: 4.197387974394147e-05,
 -23.0: 6.072816643804298e-05,
 -22.0: 7.233796296296296e-05,
 -21.0: 0.00011252572016460906,
 -20.0: 0.0001643232738911751,
 -19.0: 0.00021254858253315045,
 -18.0: 0.00029917552583447643,
 -17.0: 0.00048225308641975306,
 -16.0: 0.0006492555441243713,
 -15.0: 0.0007376686099679927,
 -14.0: 0.0012056327160493826,
 -13.0: 0.0012931527206218563,
 -12.0: 0.001684313557384545,
 -11.0: 0.0019075788751714677,
 -10.0: 0.002004922553726566,
 -9.0: 0.002404120941929584,
 -8.0: 0.0027211576931870144,
 -7.0: 0.003256101394604481,
 -6.0: 0.003409707933241884,
 -5.0: 0.004004486739826246,
 -4.0: 0.00

In [358]:
#https://gist.github.com/mreid/fdf6353ec39d050e972b
# Example Huffman coding implementation
# Distributions are represented as dictionaries of { 'symbol': probability }
# Codes are dictionaries too: { 'symbol': 'codeword' }

def huffman(p):
    '''Return a Huffman code for an ensemble with distribution p.'''
    assert(sum(p.values()) > 0.99) # Ensure probabilities sum to 1
    assert(sum(p.values()) < 1.01) # Ensure probabilities sum to 1

    # Base case of only two symbols, assign 0 or 1 arbitrarily
    if(len(p) == 2):
        return dict(zip(p.keys(), ['0', '1']))

    # Create a new distribution by merging lowest prob. pair
    p_prime = p.copy()
    a1, a2 = lowest_prob_pair(p)
    p1, p2 = p_prime.pop(a1), p_prime.pop(a2)
    p_prime[a1 + a2] = p1 + p2

    # Recurse and construct code on new distribution
    c = huffman(p_prime)
    ca1a2 = c.pop(a1 + a2)
    c[a1], c[a2] = ca1a2 + '0', ca1a2 + '1'

    return c

def lowest_prob_pair(p):
    '''Return pair of symbols from distribution p with lowest probabilities.'''
    assert(len(p) >= 2) # Ensure there are at least 2 symbols in the dist.

    sorted_p = sorted(p.items())
    return sorted_p[0][0], sorted_p[1][0]

# Example execution
# ex1 = { 'a': 0.5, 'b': 0.25, 'c': 0.25 }
# huffman(ex1) # => {'a': '0', 'c': '10', 'b': '11'}
# huffmans = [huffman(f) for f in range(len(frequencies))]


huffman_trees = [ huffman(probs[i]) for i in range(3)]
huffman_trees[0]

{-170.0: '100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000',
 -169.0: '100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001',
 -168.0: '1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [359]:
T_Y.shape

(17496, 8, 8)